In [1]:
from llama_index.vector_stores.qdrant import QdrantVectorStore
from qdrant_client import QdrantClient, AsyncQdrantClient
from llama_index.core import Document, VectorStoreIndex, StorageContext, Settings
from dotenv import load_dotenv
from llama_index.embeddings.openai import OpenAIEmbedding
from qdrant_client.http.models import PayloadSchemaType
import os
load_dotenv()

client = QdrantClient(
    url=os.getenv("QDRANT_URL"), 
    api_key=os.getenv("QDRANT_API_KEY"),
)
aclient = AsyncQdrantClient(
    url=os.getenv("QDRANT_URL"), 
    api_key=os.getenv("QDRANT_API_KEY"),
) 

def config_db(collection_name):
    Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")
    vector_store = QdrantVectorStore(
        collection_name=collection_name,
        client=client,
        aclient=aclient,
        enable_hybrid=True,
        fastembed_sparse_model="Qdrant/bm25",
        batch_size=20,
    )
    return vector_store

D:\study\LammaIndex\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [57]:
from llama_index.core.settings import Settings
# from llama_index.postprocessor.colbert_rerank import ColbertRerank
from llama_index.core.postprocessor import LLMRerank
from llama_index.core import StorageContext, VectorStoreIndex
from llama_index.core.vector_stores import (
    MetadataFilter,
    MetadataFilters,
    FilterOperator,
    FilterCondition
)
import asyncio
from concurrent.futures import ThreadPoolExecutor
from llama_index.core import QueryBundle
from llama_index.core.prompts import PromptTemplate
from llama_index.core.prompts.prompt_type import PromptType
import json

In [58]:
vector_store = config_db("hello_my_friend2")
index = VectorStoreIndex.from_vector_store(vector_store=vector_store)
filename_ids = ['f_b20befa8-8035-11f0-baa1-38f3abb08dd1']
filters_chunk = MetadataFilters(
    filters=[
        MetadataFilter(key="filename_id", operator=FilterOperator.IN, value=filename_ids),
        MetadataFilter(key="type", operator=FilterOperator.EQ, value="chunk_document"),
    ],
    condition=FilterCondition.AND,
)
query_str = "Yêu cầu với module chỉnh lưu (Rectifier): Tổng  độ  méo  hài  THD của Rectifier: ≤5% với tải từ 50÷100%"
# query_bundle = QueryBundle(query_str)
retriever_chunk = index.as_retriever(
    similarity_top_k=10,
    sparse_top_k=15,
    # verbose=True,
    enable_hybrid=True,
    filters=filters_chunk
)
retrieved_nodes = retriever_chunk.retrieve(query_str)
# reranker = LLMRerank(
#     # choice_select_prompt=custom_rerank_prompt,
#     choice_batch_size=10,
#     top_n=5,
#     llm=Settings.llm
# )
# results = reranker.postprocess_nodes(
#     retrieved_nodes, query_bundle
# )

In [59]:
content = ""
for i, result in enumerate(retrieved_nodes, start=1):
    metadata = result.metadata
    file_name = metadata["file_name"] + ".pdf"
    page = metadata["page"]
    table = metadata["table_name"]
    figure_name = metadata.get("figure_name")
    text = result.text.strip()
    content += f"""
    ========== CHUNK {metadata['chunk_id']} ========
        page: {page}
        file: {file_name}
        figure: {figure_name}
        table: {table}

        Nội dung của chunk: {text}
        ========== CHUNK {metadata['chunk_id']} END ==========\n\n"""

In [60]:
content

"\n    ========== CHUNK R48-1000e3_User_Manual_26 ========\n        page: 13\n        file: R48-1000e3_User_Manual.pdf\n        figure: None\n        table: None\n\n        Nội dung của chunk: ## Environmental Ratings:\n\n- Operating Ambient Temperature Range:\n  a) +45 °C (+113 °F) to +75 °C (+176 °F) with derating output.\n  b) -40 °C (-40 °F) to +45 °C (+113 °F) with full power performance.\n\n- Temperature Coefficient: 0.02 % per degrees Celsius.\n\n- Storage Ambient Temperature Range: -25 °C (-13 °F) to +55 °C (+131 °F).\n\n- Relative Humidity: This rectifier is capable of operating in an ambient relative humidity range of 0 % to 90 %, non-condensing.\n\n- Altitude: 2000 m (6560 ft) at full power (power limited for heights above 2000 m).\n\n- Surge Protection: Compliance with EN61000-4-5 (4kV Line to Line, 4kV Line to Earth). Capable of withstanding surges per ANSI/IEEE C 62.41 1999 Category B3 across the input terminals.\n\n> NOTE! This level of protection is a widely used standa